This notebook show an example on how to run predictions using a pre-trained a lgsegmentor.

This example it uses the validation data 

In [1]:
import numpy as np
import tifffile
import napari
import leopardgecko.segmentor as lgs
import leopardgecko.utils as lgu

import logging
logging.basicConfig(level=logging.DEBUG)

# Load data  for prediction and respective labels

In [2]:
data_to_pred = tifffile.imread("test_data/TS_0005_crop_val.tif")
labels_to_compare = tifffile.imread("test_data/TS_0005_ribos_membr_crop_val.tif")

In [3]:
print(data_to_pred.shape)

(256, 256, 256)


# Setup lg segmentor and run

In [4]:
with lgs.cMultiAxisRotationsSegmentor(lgmodel_fn="2023-09-29_ribos_membr.lgsegmod") as lgsegmentor0:
    print(lgsegmentor0)
    #Calculates consistency score and volume segmantics. Output filename must be h5 file
    lgsegmentor0.set_cuda_device(1)
    pred_labels = lgsegmentor0.predict(data_to_pred)

DEBUG:root:cMultiAxisRotationsSegmentor __init__() with temp_data_outdir:None , cuda_device:0
DEBUG:root:load_model()
DEBUG:root:predict() data_in.shape:(256, 256, 256), data_in.dtype:int8, use_dask:False
INFO:root:Setting up NN1 prediction
DEBUG:root:clear()
DEBUG:root:NN1_predict()
DEBUG:root:data_to_predict not a list. Converting to list
INFO:root:number of data sets to predict: 1
INFO:root:Data to predict index:0
DEBUG:root:VolSeg2DPredictionManager.__init__()
DEBUG:root:BaseDataManager.__init__()
DEBUG:root:BaseDataManager._preprocess_data()
INFO:root:Calculating mean of data...
INFO:root:Mean value: 3.376452088356018
DEBUG:root:clip_to_uint8()
INFO:root:Calculating standard deviation.
INFO:root:Std dev: 22.28497505498702. Calculating stats.
INFO:root:Lower bound: -54.00735867823556, upper bound: 60.7602628549476


INFO:root:Number of voxels above upper bound to be clipped 95283 - percentage 0.568%
INFO:root:Number of voxels below lower bound to be clipped 175185 - percentage 1.044%
INFO:root:Rescaling intensities.
INFO:root:Data is already in integer dtype, converting to float for rescaling.
INFO:root:Converting to uint8.
DEBUG:root:VolSeg2dPredictor.__init__() ,settings.cuda_device:1
INFO:root:Loading model dictionary from file.
INFO:root:Sending the U-Net model to device 1
INFO:root:Loading in the saved weights.
DEBUG:root:clear()
INFO:root:Volume to be rotated by 0 degrees
INFO:root:Predicting YX slices:
DEBUG:root:_predict_single_axis_all_probs() with axis:Axis.Z
DEBUG:root:rotate_array_to_axis()
INFO:root:Free GPU memory is 31.66 GB. Batch size will be 4.
INFO:root:Predicting segmentation for volume of shape (256, 256, 256).
Prediction batch: 100%|██████████████████████████████| 64/64 [00:02<00:00, 24.53it/s]
INFO:root:Completed prediction. Now manipulating result before returning.
DEBUG:ro

[########################################] | 100% Completed | 25.38 ss


INFO:root:NN2 prediction complete.


# Visualise result

In [5]:
import leopardgecko.utils as lgu
data_cscore = lgu.read_h5_to_np("./consistency_score.h5")
pred_vs = lgu.read_h5_to_np("./VS_prediction.h5")

In [6]:
import napari
NV=napari.Viewer()
NV.add_image(data_to_pred)
NV.add_labels(labels_to_compare)
NV.add_labels(pred_labels, color={1:'green'})
NV.add_labels(pred_vs, color={1:'blue'})
NV.add_image(data_cscore)

DEBUG:in_n_out:Registering provider of <class 'napari.viewer.Viewer'>: <function _provide_viewer at 0x7f5ff9168310> (weight: 0, subclassable: True)
DEBUG:in_n_out:Registering provider of <class 'napari.layers.base.base.Layer'>: <function _provide_active_layer at 0x7f5ff91681f0> (weight: 0, subclassable: True)
DEBUG:in_n_out:Registering provider of <class 'napari.components.layerlist.LayerList'>: <function _provide_active_layer_list at 0x7f5ff9168280> (weight: 0, subclassable: True)
DEBUG:in_n_out:Registering processor of <function NewType.<locals>.new_type at 0x7f5ff91f8430>: <function _add_layer_data_tuples_to_viewer at 0x7f5ff9168430> (weight: 0, subclassable: False)
DEBUG:in_n_out:Registering processor of typing.List[LayerDataTuple]: <function _add_layer_data_tuples_to_viewer at 0x7f5ff9168430> (weight: 0, subclassable: False)
DEBUG:in_n_out:Registering processor of <class 'napari.layers.base.base.Layer'>: <function _add_layer_to_viewer at 0x7f5ff9168160> (weight: 0, subclassable: T

<Image layer 'data_cscore' at 0x7f5dc05f5dc0>

# Compare against ground truth

Use leopardgecko metrics to get voxel-F1 (dice score)

In [7]:
import leopardgecko.metrics as lgm

In [8]:
lgm.MetricScoreOfVols_Dice(pred_labels,labels_to_compare)

INFO:root:MetricScoreOfVols_Dice()
INFO:root:Number of class segmentations in first volume 3
INFO:root:Number of class segmentations in second volume 3
INFO:root:Calculating using numpy
INFO:root:dicescores = {1: 0.5188051691479704, 2: 0.5507384751326172}


(0.5347718221402937, {1: 0.5188051691479704, 2: 0.5507384751326172})

DEBUG:napari.loader:ImageSlice.__init__
DEBUG:napari.loader:ImageSlice.__init__
DEBUG:napari.loader:ImageSlice.__init__
DEBUG:napari.loader:ImageSlice.__init__
DEBUG:napari.loader:ImageSlice.__init__


dice score 0.535